# 线性回归 简洁实现

首先 我们导入必要的库函数

ps:torch.utils.data 模块提供了数据加载到模型的工具，如 Dataset 和 DataLoader 类，帮助用户高效地迭代和处理大型数据集。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.utils import data
import numpy as np

In [6]:
true_b = torch.tensor([3])
true_w = torch.tensor([5.2, 3.1])
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
true_w, true_b

(tensor([5.2000, 3.1000]), tensor([3]))

In [105]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 1.3222,  0.0063],
         [ 0.4284,  0.8522],
         [ 0.6102, -1.3927],
         [-0.8808,  0.1120],
         [-0.3422,  0.7647],
         [-1.4106, -0.7840],
         [ 0.2117,  0.0544],
         [-0.4227,  0.4994],
         [-2.0795, -0.6959],
         [-0.3908,  0.4869]]),
 tensor([[ 9.8852],
         [ 7.8553],
         [ 1.8590],
         [-1.2408],
         [ 3.6040],
         [-6.7581],
         [ 4.2801],
         [ 2.3587],
         [-9.9696],
         [ 2.4694]])]

In [33]:
import torch.nn as nn
# 创建一个简单的序列模型
model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 2)
)

#从 PyTorch 1.2 开始，你还可以在 nn.Sequential 中使用 OrderedDict 来给每一层指定一个名称，这有助于未来对模型的每一层进行访问或修改：
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
    ('linear1', nn.Linear(10, 5)),
    ('relu', nn.ReLU()),
    ('linear2', nn.Linear(5, 2))
]))

# 访问第一个线性层的权重
print(model.linear1.weight)

#你也可以先创建一个空的 nn.Sequential 对象，然后逐一添加层：
model = nn.Sequential()
model.add_module('linear1', nn.Linear(10, 5))
model.add_module('relu', nn.ReLU())
model.add_module('linear2', nn.Linear(5, 2))

#一旦模型被定义，就可以像使用普通的 PyTorch 模型一样使用它，包括传递输入数据进行前向传播：
import torch
# 创建输入张量
input_tensor = torch.randn(1, 10)  # 假设批量大小为1
# 使用模型进行前向传播
output = model(input_tensor)
print(output)


Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([[-0.9931,  1.5926,  0.0965, -0.6080,  2.2544,  0.2710,  0.0412, -0.5330,
         -0.0174,  1.7859]])
tensor(3.8902)
tensor([[3.8902, 3.8902, 3.8902, 3.8902, 3.8902]], grad_fn=<AddmmBackward0>)


In [106]:
model = nn.Sequential(nn.Linear(2,  1))
model[0].weight.data.normal_(3, 1000)
model[0].bias.data.fill_(0)
model[0].weight, model[0].bias

trainer = torch.optim.SGD(model.parameters(), lr=0.03)
loss = nn.MSELoss()

In [107]:
num_epoch = 10
for i in range(num_epoch):
    for x, y in data_iter:
        l = loss(model(x), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(model(features), labels)
    print(f'epoch {i + 1}, loss {l:f}')
print(model[0].weight.data, model[0].bias.data)

epoch 1, loss 0.141492
epoch 2, loss 0.000094
epoch 3, loss 0.000095
epoch 4, loss 0.000095
epoch 5, loss 0.000094
epoch 6, loss 0.000094
epoch 7, loss 0.000094
epoch 8, loss 0.000094
epoch 9, loss 0.000095
epoch 10, loss 0.000095
tensor([[5.1995, 3.0989]]) tensor([3.0003])
